quick start: http://docs.h5py.org/en/latest/quick.html

Dimension Scales: http://docs.h5py.org/en/latest/high/dims.html

In [ ]:
import h5py

In [ ]:
f = h5py.File('test.hdf5', mode='w')  # can also be used as context manager

In [ ]:
f.name

In [ ]:
f['a'] = 12

In [ ]:
list(f.keys())

In [ ]:
data = f['a']
data

In [ ]:
data[...]

In [ ]:
data.shape

In [ ]:
list(data.dims)

In [ ]:
data.dtype

In [ ]:
data.maxshape

In [ ]:
data.name

In [ ]:
data.attrs['extra info'] = 'super data'

In [ ]:
dict(data.attrs.items())

In [ ]:
dict(f.items())

Dimension Scales

http://docs.h5py.org/en/latest/high/dims.html

In [ ]:
import numpy as np

In [ ]:
numpy_array = np.array([[1., 2, 3], [4, 5, 6]])

In [ ]:
f['b'] = numpy_array
f['b']

In [ ]:
f.create_dataset('c', dtype='float32', data=numpy_array)

In [ ]:
dict(f.items())

In [ ]:
f['b'].dims[1].label = 'nice second dimension!'

In [ ]:
f['myscale'] = [1, 2, 3, 4]

In [ ]:
# this seems to be only necessary for setting a label:
f['c'].dims.create_scale(f['myscale'], 'my scale label')
#h5py.h5ds.set_scale(f['myscale'].id, b'my scale label')

In [ ]:
f['c'].dims[0].attach_scale(f['myscale'])

In [ ]:
[dim.label for dim in f['b'].dims]

In [ ]:
list(f['c'].dims.keys())

In [ ]:
list(f['c'].dims[0].keys())

In [ ]:
f['c'].dims[0][0]

In [ ]:
f['c'].dims[0]['my scale label']

In [ ]:
f.close()

In [ ]:
!h5dump test.hdf5

In [ ]:
with h5py.File('two_scales.hdf5', mode='w') as f:
    f['a'] = [1, 2, 3]
    f['scale1'] = 1
    f['scale2'] = 2
    f['a'].dims.create_scale(f['scale1'], 'scale one')
    h5py.h5ds.set_scale(f['scale2'].id, b'scale two')
    f['a'].dims[0].attach_scale(f['scale1'])
    f['a'].dims[0].attach_scale(f['scale2'])

In [ ]:
!h5dump two_scales.hdf5